In [1]:
from greensight.utils import DIR_DATA
import pandas as pd
import numpy as np
import json
import pandas as pd

In [2]:
# load DataFrame
# s2_path = DIR_DATA / "processed_data/sentinel_two_indices_df.hdf"
s2_path = DIR_DATA / "processed_data/sentinel_two_indices_df_droppedna.hdf"

df_data = pd.read_parquet(s2_path)
df_orig = df_data.copy()
df_data

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                  msavi          pvi         mcari           bi     ndwi2   
2017-04-01        94.486726  3937.384134  65370.597888 -2951.914648 -0.493598   
2017-05-01       121.030270  4135.544158  51998.382445 -3145.115897 -0.482986   
2017-06-01        92.271443  4150.948900  40180.548951 -2950.267034 -0.493938   
2017-07-01        15.527558  3789.197962  38911.867444 -2696.027001 -0.464355   
2017-08-01              NaN          NaN           NaN          NaN       NaN   
...                     ...          ...           ...          ...       ...   
2024-08-01       217.549152  3360.679176  18173.069533 -1971.408288 -0.458801   
2024-09-01       172.578872  3293.176882  18559.690544 -1995.008918 -0.464086   
2024-10-01       131.172695  3128.474663  23134.724415 -2060.726741 -0.439133   
2024-11-01       258.295717  2709.468640  19269.664957 -1753.927588 -0.436032   
2024-12-01        50.240398  2530.506753  15761.605438 -1607.023690 -0.429346   

greenbelt                                                             ...  \
shape                                                                 ...   
band            msavi2     s2rep         wdvi     gndvi          bi2  ...   
2017-04-01   94.486726  0.337793  3957.022082  0.698364 -2951.914648  ...   
2017-05-01  121.030270  0.359582  4156.170441  0.701280 -3145.115897  ...   
2017-06-01   92.271443  0.295201  4171.652016  0.670956 -2950.267034  ...   
2017-07-01   15.527558  0.251190  3808.096822  0.614131 -2696.027001  ...   
2017-08-01         NaN       NaN          NaN       NaN          NaN  ...   
...                ...       ...          ...       ...          ...  ...   
2024-08-01  217.549152  0.211111  3377.440772  0.602442 -1971.408288  ...   
2024-09-01  172.578872  0.213965  3309.601806  0.605956 -1995.008918  ...   
2024-10-01  131.172695  0.259195  3144.078124  0.615078 -2060.726741  ...   
2024-11-01  258.295717  0.240204  2722.982283  0.599835 -1753.927588  ...   
2024-12-01   50.240398  0.239063  2543.127812  0.597630 -1607.023690  ...   

greenbelt       York                                                    \
shape      E06000014                                                     
band            ndvi      ndwi      savi      ipvi        ci      gemi   
2017-04-01  0.419859 -0.434996  0.629708  0.328669  0.419859 -0.831839   
2017-05-01  0.754145 -0.699359  1.131079  0.671939  0.754145 -0.431212   
2017-06-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-07-01       NaN       NaN       NaN       NaN       NaN       NaN   
2017-08-01  0.621914 -0.604316  0.932758  0.527854  0.621914 -0.369069   
...              ...       ...       ...       ...       ...       ...   
2024-08-01  0.549143 -0.544992  0.823604  0.459687  0.549143 -0.388129   
2024-09-01  0.564789 -0.562622  0.847057  0.480541  0.564789 -0.267417   
2024-10-01  0.588446 -0.560214  0.882515  0.493619  0.588446 -0.361902   
2024-11-01  0.618103 -0.598243  0.926893  0.523736  0.618103 -0.375574   
2024-12-01  0.590813 -0.548785  0.886046  0.490100  0.590813 -0.395506   

greenbelt                            Burton upon Trent and Swadlincote         
shape                                                        E07000193         
band              msavi       msavi2                             msavi msavi2  
2017-04-01          NaN          NaN                               NaN    NaN  
2017-05-01   751.027167   751.027167                               NaN    NaN  
2017-06-01          NaN          NaN                               NaN    NaN  
2017-07-01          NaN          NaN                               NaN    NaN  
2017-08-01          NaN          NaN                               NaN    NaN  
...                 ...          ...                               ...    ...  
2024-08-01  1025.252629  1025

In [3]:
lookup_path = DIR_DATA / "id_lookup/id_lookup.json"
with open(lookup_path, "r") as in_file:
    D_lookup = json.load(in_file)

In [4]:
# create weighting dataframe

# instantiate empty df
df_weights = pd.DataFrame(np.nan, index=df_data.index, columns=df_data.columns )

# apply weighting
for code, dct in D_lookup.items():
    # print(code)
    area = dct["Area_ha"]

    greenbelt_name = D_lookup[code] ["GB_Name"]
    greenbelt_area = float(np.sum([val["Area_ha"] for val in D_lookup.values() if val["GB_Name"] == greenbelt_name]))

    df_weights.loc[:, (slice(None), code, slice(None))] = area/greenbelt_area


df_weights.columns = pd.MultiIndex.from_tuples([(i[0].replace("-", " "), i[1], i[2]) for i in df_weights.columns])
df_weights.columns.names = ["greenbelt", "shape", "band"]
df_weights.head()

greenbelt  Bath and Bristol                                                    \
shape             E06000022                                                     
band                  msavi       pvi     mcari        bi     ndwi2    msavi2   
2017-04-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-05-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-06-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-07-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   
2017-08-01         0.345098  0.345098  0.345098  0.345098  0.345098  0.345098   

greenbelt                                           ...      York            \
shape                                               ... E06000014             
band           s2rep      wdvi     gndvi       bi2  ...      ndvi      ndwi   
2017-04-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-05-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-06-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-07-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   
2017-08-01  0.345098  0.345098  0.345098  0.345098  ...  0.801384  0.801384   

greenbelt                                                               \
shape                                                                    
band            savi      ipvi        ci      gemi     msavi    msavi2   
2017-04-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-05-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-06-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-07-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   
2017-08-01  0.801384  0.801384  0.801384  0.801384  0.801384  0.801384   

greenbelt  Burton upon Trent and Swadlincote         
shape                              E07000193         
band                                   msavi msavi2  
2017-04-01                               1.0    1.0  
2017-05-01                               1.0    1.0  
2017-06-01                               1.0    1.0  
2017-07-01                               1.0    1.0  
2017-08-01                               1.0    1.0  

[5 rows x 5400 columns]

In [5]:
# check all greenbelts for each band sum to 1.0
assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()

/tmp/ipykernel_347628/4025412648.py:2: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  assert np.isclose(df_weights.groupby(level=["greenbelt", "band"], axis=1).sum(), 1.0, atol=1.e-6).all().all()
/tmp/ipykernel_347628/4025412648.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df_weights.groupby(level=["greenbelt", "band"], axis=1).sum()


greenbelt  Bath and Bristol                                                  \
band                   arvi   bi  bi2   ci  dvi gemi gndvi ipvi ireci mcari   
2017-04-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-05-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-06-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-07-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2017-08-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
...                     ...  ...  ...  ...  ...  ...   ...  ...   ...   ...   
2024-08-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-09-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-10-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-11-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   
2024-12-01              1.0  1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0   

greenbelt   ...  York                                                  
band        ... pssra  pvi reip   ri  rvi s2rep savi tndvi tsavi wdvi  
2017-04-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-05-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-06-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-07-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2017-08-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
...         ...   ...  ...  ...  ...  ...   ...  ...   ...   ...  ...  
2024-08-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-09-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-10-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-11-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  
2024-12-01  ...   1.0  1.0  1.0  1.0  1.0   1.0  1.0   1.0   1.0  1.0  

[93 rows x 480 columns]

In [6]:
# save DataFrame
# output_path = DIR_DATA / "processed_data/sentinel_two_index_weights.hdf"
output_path = DIR_DATA / "processed_data/sentinel_two_index_weights_droppedna.hdf"
df_weights.to_parquet(output_path)